In [1]:
import statlist
from item_simulation import ItemTest
from numpy import random
from copy import copy

Now for each item you need to make a class inherited from __ItemTest__ and define two methods for it: constructor and roll_stats. Below is example of trifecta Hellfire Amulet for wizard defined that way

In [2]:
class Hellfire(ItemTest):
    def __init__(self):
        self.isoffhand = False
        self.isweapon = False
        # here in most cases you just need to pick the right type, but sometimes you also need to edit these dicts
        # you only need to add non-standard stats here if they are rerollable
        self.primary_stats = copy(statlist.amulet_primary)
        self.secondary_stats = copy(statlist.amulet_secondary)
        # number of primary and secondary stats that this item can roll
        self.n = 2
        self.m = 1
        # all guaranteed affixes, you can put any name here it doesn't have to be one of the affixes from statlist
        # like put "orange" or "yangrcr" for any legendary affixes it doesn't really matter
        self.guaranteed = ['mainstat', 'socket', 'passive']
        # stats that you want and how much of each you want
        # other than stats from self.guaranteed and statlist, you can put "ancient" and "primal" here
        # you can define multiple configurations if you want
        # the minimum value of stat is a bit tricky
        # 0 (or any negative number) means you don't care (usually the case with mainstat)
        # any other number for most stats is self explanatory, e.g. 'CC':9 means you want at least 9% crit chance
        # but %Skill, Secondary Resist and %Element are combined with other similar stats 
        # for example here we have stat 'element' which is actually 4 stats: arcane, cold, fire, lightning
        # each of them can roll in 15-20 range (6 points), so we roll all 4 at once as 1-24 range
        # this way we can say that 1-18 correspond to 3 elements that we don't need, and 19-24 is our desired element
        # in this case arcane. 19 = 15% aracne, 24 = 20% arcane. Same with 'passive' affix: we say that each of 18
        # wizard's passives is a number in range 1-18 and then the last 5 (14-18) are the good passives
        self.desired = [
                        {'passive':14, 'element':20, 'CC':9, 'CHD':90, 'socket':0, 'ancient':0},
                        {'passive':14, 'element':23, 'CC':8, 'CHD':85, 'socket':0, 'ancient':0},
                        {'passive':14, 'element':19, 'CC':10, 'CHD':95, 'socket':0, 'ancient':0},
                       ]

    # this methods needs to produce a dict with affix - value for each affix that occurs in self.desired
    def roll_stats(self, isprimal):
        # this line will deal with all stats where you put 0 as value, UNLESS your different configurations in
        # self.desired have different such stats
        rolls = dict([(key, 0) for key in self.desired[0]])
        # rolling every stats which value actually matters, most are self-explanatory multiple affixes are covered above
        rolls['CC'] = random.randint(16, 21) * 0.5
        rolls['CHD'] = random.randint(51, 101)
        rolls['element'] = random.randint(1, 25)
        rolls['passive'] = random.randint(1, 19)
        if isprimal:
            # for primal set rolls to their max value, for multistats you have to randomly choose one of max values
            rolls['CC'] = 10
            rolls['CHD'] = 100
            rolls['element'] = random.randint(1, 5) * 6
            rolls['passive'] = random.randint(1, 19)
            
        return rolls

After that you just need to create an instance of your class and call its __test()__ method,

test takes 2 arguments: number of tries (default 100000) and debug (default false)

In [3]:
Hellfire().test(int(4e5))

Simulating: [--------------------------------------->] 100.00%

On average, one in 5714 items has desired stats.


In [4]:
Hellfire().test(tries=20, debug=True)

['mainstat', 'socket', 'passive', 'RCR', 'AR', 'level']
{'passive': 1, 'element': 16, 'CC': 8.5, 'CHD': 88, 'socket': 0, 'ancient': 0}
['mainstat', 'socket', 'passive', 'CC', 'element', 'res']
{'passive': 12, 'element': 10, 'CC': 9.0, 'CHD': 66, 'socket': 0, 'ancient': 0}
['mainstat', 'socket', 'passive', 'avgdmg', 'element', 'LPK']
{'passive': 17, 'element': 12, 'CC': 8.5, 'CHD': 82, 'socket': 0, 'ancient': 0}
['mainstat', 'socket', 'passive', 'element', 'AD', 'globe']
{'passive': 11, 'element': 15, 'CC': 10.0, 'CHD': 92, 'socket': 0, 'ancient': 0}
['mainstat', 'socket', 'passive', 'AD', 'vit', 'CCR']
{'passive': 11, 'element': 1, 'CC': 9.0, 'CHD': 64, 'socket': 0, 'ancient': 0}
['mainstat', 'socket', 'passive', 'element', 'CDR', 'res']
{'passive': 10, 'element': 4, 'CC': 8.0, 'CHD': 58, 'socket': 0, 'ancient': 0}
['mainstat', 'socket', 'passive', 'element', 'CDR', 'exp']
{'passive': 2, 'element': 11, 'CC': 8.5, 'CHD': 99, 'socket': 0, 'ancient': 0}
['mainstat', 'socket', 'passive', '

Some more examples of items for copypasting *etc*.

In [5]:
# here I wanted to gamble/upgrade bracers for lwiz and tried to figure out what class is best for that
# wiz has a chance to roll the right element, while necro gets 25% more APDs from the same amount of shards 
class APD(ItemTest):
    def __init__(self):
        self.isoffhand = False
        self.isweapon = False
        self.primary_stats = copy(statlist.bracer_primary)
        self.secondary_stats = copy(statlist.bracer_secondary)
        self.n = 3
        self.m = 1
        self.guaranteed = ['mainstat', 'reduction']
        self.desired = [
                        {'CC':5.5, 'mainstat':0, 'vit':0, 'element':19, 'ancient':0, 'ranged':0, 'reduction':11},
                        {'CC':5.5, 'mainstat':0, 'LPH':0, 'element':19, 'ancient':0, 'reduction':11},
                       ]
        
    def roll_stats(self, isprimal):
        rolls = dict([(key, 0) for key in self.desired[0]]) 
        rolls['CC'] = random.randint(9, 13) * 0.5
        rolls['element'] = random.randint(1, 25)
        rolls['reduction'] = random.randint(9, 13)
        if isprimal:
            rolls['CC'] = 6
            rolls['element'] = random.randint(1, 5) * 6
            rolls['reduction'] = 12
        return rolls
        
class APD_necro(ItemTest):
    def __init__(self):
        self.isoffhand = False
        self.isweapon = False
        self.primary_stats = copy(statlist.bracer_primary)
        self.secondary_stats = copy(statlist.bracer_secondary)
        self.n = 3
        self.m = 1
        self.guaranteed = ['mainstat', 'reduction']
        self.desired = [
                        {'CC':5.5, 'mainstat':0, 'vit':0, 'element':19, 'ancient':0, 'ranged':0, 'reduction':11},
                        {'CC':5.5, 'mainstat':0, 'LPH':0, 'element':19, 'ancient':0, 'reduction':11},
                       ]
        
    def roll_stats(self, isprimal):
        rolls = dict([(key, 0) for key in self.desired[0]]) 
        rolls['CC'] = random.randint(9, 13) * 0.5
        # the only diffectemce here is that necro can only roll element in range 1-18, and we need 19+ (i.e. lightning)
        rolls['element'] = random.randint(1, 19)
        rolls['reduction'] = random.randint(9, 13)
        if isprimal:
            rolls['CC'] = 6
            rolls['element'] = random.randint(1, 4) * 6
            rolls['reduction'] = 12
        return rolls

In [6]:
APD().test()
APD_necro().test()

Simulating: [--------------------------------------->] 99.96%

On average, one in 157 items has desired stats.
Simulating: [--------------------------------------->] 99.96%

On average, one in 676 items has desired stats.


So yeah, apparently it's much better on wiz LuL

In [7]:
class Yang(ItemTest):
    def __init__(self):
        self.isoffhand = False
        self.isweapon = True
        self.primary_stats = copy(statlist.weapon_primary)
        self.secondary_stats = copy(statlist.bow_secondary)
        self.n = 1
        self.m = 1
        self.guaranteed = ['mainstat', 'damage', 'orange', 'dmg', 'yangrcr']
        # for damage I just use a single value - min + max, it goes up to 3500, and I want 3370+
        # so this bow will be within 90 DPS of Primal
        self.desired = [
                        {'mainstat':0, 'damage':3370, 'dmg':8, 'orange':197, 'yangrcr':46, 'resource':12, 'AD':22, 'ancient':0},
                        {'mainstat':0, 'damage':3370, 'dmg':8, 'orange':197, 'yangrcr':46, 'resource':12, 'CDR':8, 'ancient':0},
                        {'mainstat':0, 'damage':3370, 'dmg':8, 'orange':197, 'yangrcr':46, 'resource':12, 'LPH':0, 'ancient':0},
                        {'mainstat':0, 'damage':3370, 'dmg':8, 'orange':197, 'yangrcr':46, 'resource':12, 'elitedmg':9, 'ancient':0},
                       ]
        
    def roll_stats(self, isprimal):
        rolls = dict([(key, 0) for key in self.desired[0]]) 
        rolls['damage'] = random.randint(1318, 1561) + random.randint(1609, 1941)
        rolls['dmg'] = random.randint(6, 11) 
        rolls['yangrcr'] = random.randint(41, 51)
        rolls['CDR'] = random.randint(6, 11)
        rolls['AD'] = random.randint(16, 25) 
        rolls['elitedmg'] = random.randint(6, 11)
        rolls['resuorce'] = random.randint(10, 13) 
        rolls['orange'] = random.randint(151, 201)
        if isprimal:
            rolls['damage'] = 3500
            rolls['dmg'] = 10
            rolls['yangrcr'] = 50
            rolls['CDR'] = 10
            rolls['AD'] = 24
            rolls['elitedmg'] = 10
            rolls['resource'] = 12
            rolls['orange'] = 200
        return rolls

In [8]:
class Cindercoat(ItemTest):
    def __init__(self):
        self.isoffhand = False
        self.isweapon = False
        # here we need to add unique rerollable stat fire damage on chest piece
        # since it's a guaranteed stat, its weight doesn't really matter, I put 1000 here but anything will do
        self.primary_stats = {**statlist.chest_primary, 'fire':1000}
        self.secondary_stats = copy(statlist.chest_secondary)
        self.n = 2
        self.m = 1
        self.guaranteed = ['mainstat', 'fire', 'firercr']
        self.desired = [
                        {'ancient':0, 'mainstat':0, 'vit':0, 'socket':3, 'fire':20, 'firercr':29, 'res':0},
                       ]
        
    def roll_stats(self, isprimal):
        rolls = dict([(key, 0) for key in self.desired[0]]) 
        rolls['fire'] = random.randint(15,21)
        rolls['firercr'] = random.randint(23,31)
        rolls['sockets'] = random.randint(1,4)
        if isprimal:
            rolls['fire'] = 20
            rolls['firercr'] = 30
            rolls['sockets'] = 3
        return rolls

In [9]:
#Swamp Land Wader with Holy%

class SLW(ItemTest):
    def __init__(self):
        self.isoffhand = False
        self.isweapon = False
        # once again add new rerollable stat, this time weight 0 since it doesn't matter
        self.primary_stats = {**statlist.pants_primary, "element":0}
        self.secondary_stats = copy(statlist.pants_secondary)
        self.n = 2
        self.m = 1
        self.guaranteed = ['mainstat', 'element', 'socket']
        self.desired = [
                        {'socket':2, 'element':2, 'mainstat':0, 'vit':0, 'AR':0, 'ancient':0},
                       ]
        
    def roll_stats(self, isprimal):
        rolls = dict([(key, 0) for key in self.desired[0]]) 
        # element always rolls 1, but we need 2 - SLWs only come with WD elements
        rolls['element'] = 1
        rolls['socket'] = 2
        if isprimal:
            # here primal changes nothing as we don't care about any rolls
            pass
        return rolls